In [22]:
import pickle
import numpy as np
import dnn_model
import torch

In [26]:
DATA_FILE = '../data/data_large_multi_mpc(1).pkl'
MODEL_DIMENSIONS = [39, 128, 128, 64, 1]
OPPONENTS = {
    0: [2, 1],
    1: [2, 0],
    2: [0, 1]
}
HIDDEN_WINDOW = 250
DISCOUNT_FACTOR = 0.99

data = pickle.load(open(DATA_FILE, 'rb'))

In [21]:
# data = np.array(data[:-1])  # Incomplete last episode
data.shape

(45, 500, 39)

In [24]:
def fix_difference(t):
    t = (t > 75.0435) * (t - 150.087) + (t < -75.0435) * (t + 150.087) + (t <= 75.0435) * (t >= -75.0435) * t
    return t

In [28]:
X = data[:,:,:].copy()
X[:,:,0] = data[:,:,2] - data[:,:,1]
X[:,:,1] -= data[:,:,0]
X[:,:,2] -= data[:,:,0]
for i in range(3):
    X[:,:,i] = fix_difference(X[:,:,i])

X = torch.tensor(X, dtype=torch.float32)

Y = torch.zeros((3, X.shape[0], X.shape[1]-1))
for i in range(3):
    Y[i, :, :] = torch.maximum(X[:, 1:, OPPONENTS[i][0]], X[:, 1:, OPPONENTS[i][1]]) - torch.maximum(X[:, :-1, OPPONENTS[i][0]], X[:, :-1, OPPONENTS[i][1]])
    Y[i, :, :] = fix_difference(Y[i, :, :])

rewards = torch.zeros(3, X.shape[0], HIDDEN_WINDOW)
masks = torch.zeros(3, X.shape[0], HIDDEN_WINDOW)

for j in range(3):
    for i in range(0, HIDDEN_WINDOW):
        rewards[j, :, :] += (DISCOUNT_FACTOR**i) * Y[j, :, i:i+X.shape[1]-HIDDEN_WINDOW] 

    masks[j, :, :] = torch.tensor((Y[j, :, :HIDDEN_WINDOW] < 600.) * (Y[j, :, :HIDDEN_WINDOW] >= -100.))

/tmp/ipykernel_195236/652533515.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks[j, :, :] = torch.tensor((Y[j, :, :HIDDEN_WINDOW] < 600.) * (Y[j, :, :HIDDEN_WINDOW] >= -100.))
